In [ ]:
import numpy as np
import pandas as pd
import itertools
from mass_function import MassFunction


In [ ]:
ALPHA = 0.1


In [ ]:
ROOT = ".."
df_data = pd.read_csv(f"{ROOT}/data/HEA_data.demo.csv", index_col=0)
ELEMENTS = np.array([
    "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu",
    "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag"])


In [ ]:
#結果の保存
df_similarity = pd.read_csv("data_executed/elements_similarity.csv", index_col=[0])
df_dissimilarity = pd.read_csv("data_executed/elements_dissimilarity.csv", index_col=[0])
df_unknown = pd.read_csv("data_executed/elements_unknown.csv", index_col=[0])


In [ ]:
index = df_similarity.index.tolist()
index[0] = ""
df_similarity.index = index


In [ ]:
columns = df_similarity.columns.tolist()
columns[0] = ""
df_similarity.columns = columns

In [ ]:
df_similarity

In [ ]:
from ERS_misc import plot_similarity_matrix
plot_similarity_matrix(df_similarity, ELEMENTS)


In [ ]:
SCALE = .1


In [ ]:
from ERS_misc import extract_HEA_evidence


In [ ]:
df_similarity

In [ ]:
candidates = [
    ["Fe", "Mn", "Co", "Ti"], ["Fe", "Mn", "Co", "Cu"], ["Fe", "Mn", "Co", "V"],
    ["Fe", "Mn", "Co", "Cr"], ["Fe", "Mn", "Co", "Ni"]]


In [ ]:
df_data

In [ ]:
final_decisions = []
for candidate in candidates:
    final_decision = MassFunction(coreset={"HEA", "not_HEA"})
    for ith, row in df_data.iterrows():
        alloy_k = ELEMENTS[row[ELEMENTS] == 1]
        label_alloy_k = row["Label"]
        evidence = extract_HEA_evidence(frozenset(candidate), 
                                        frozenset(alloy_k))
        if evidence is not None:
            combination_t, combination_v = evidence
            print(alloy_k, combination_t, combination_v)
            
            index_t, index_v = "|".join(sorted(combination_t)),\
                               "|".join(sorted(combination_v))
            similar_score = df_similarity.loc[index_t, index_v]
            mass_function = MassFunction(source=[({label_alloy_k},
                                     similar_score*SCALE)],
                                         coreset={"HEA", "not_HEA"})
            final_decision = final_decision.combine(mass_function)
    final_decisions.append(final_decision)


In [ ]:
from ERS_misc import plot_final_decisions
import matplotlib.pyplot as plt
plot_final_decisions(candidates, final_decisions)
plt.tight_layout()
